In [1]:
import logging
import os
import sys
import warnings
import shutil
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy.stats as sc_sts
import mlxtend

%matplotlib inline

In [2]:
## Some display settings
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',50)
warnings.filterwarnings(category=FutureWarning,action='ignore')

In [3]:
## Visualization Global Variables
label_dict = {'family':'Calibri','size':21,'style':'oblique','color':'coral'}
title_dict = {'family':'Calibri','size':23,'style':'oblique','color':'magenta'}
wedge_dict = {'linewidth': 1, 'edgecolor': 'black'}
txt_dict = {'family':'Calibri','size':16,'style':'oblique','color':'k'}

In [8]:
cer_cancer_df = pd.read_csv('Datasets/Cervical_cancer/kag_risk_factors_cervical_cancer.csv').rename(columns={'Citology':'Cytology'})

In [9]:
cer_cancer_df.columns

Index(['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Cytology', 'Biopsy'],
      dtype='object')